# LLM refinement with langchain

In [8]:
import os
import re
import json
import importlib
import chromadb
import pickle as pkl
import pandas as pd
from dotenv import dotenv_values
from langchain_chroma import Chroma
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain.chat_models import ChatOpenAI
from langchain_voyageai import VoyageAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
import sys

sys.path.append('../src/')

In [2]:
for i,v in dotenv_values().items():
    globals()[i]=v 

In [4]:
RESOURCES_DIR="../../resources"
llm = init_chat_model("o4-mini", model_provider="openai")
# llm = ChatOllama(model="llama3.1:8b")
embeddings_model = VoyageAIEmbeddings(model="voyage-3")

Texto a probar

In [5]:
test_df = pd.read_csv(os.path.join(PROJECT_DIR, "./datasets/TFM_test.csv"))
test_df.annotations = test_df.annotations.apply(eval)
clinical_note = test_df.texts.loc[1]
hpo_codes = test_df.annotations.iloc[1]
clinical_note

"Un hombre de 32 años acudió a una unidad regional de cirugía general por anemia sintomática grave. En un lapso de 4 meses, su hemoglobina descendió 62 puntos desde 137 gL (N: 135-180 gL). Su historial médico incluía epilepsia con valproato, arritmia cardiaca con flecainida, síndrome del túnel carpiano y compresión de la raíz nerviosa C5. No fuma, consume poco alcohol, tiene dos perros en casa y dos hijos. Sus antecedentes familiares incluían a su abuelo, que tuvo cáncer de intestino diagnosticado a los 78 años, y una abuela con cáncer de mama. La primera modalidad de diagnóstico por imagen fue un TAC que mostraba una gran masa fungiforme en el cuerpo del estómago que se extendía hasta el píloro y la primera parte del duodeno. Se sometió a una endoscopia digestiva alta (EDA) para investigar la patología. La histopatología arrojó con frecuencia un resultado de tejido hiperplásico benigno. Los primeros resultados histopatológicos informaron de mucosa gástrica con hiperplasia foveolar men

Definir el formato del input

In [9]:
chroma_client = chromadb.HttpClient(host='localhost', port=8001)

In [22]:
vectordb_sep = Chroma(client=chroma_client, embedding_function=embeddings_model, 
                  collection_name="hpo_ontology_esp_SEP")
ontology = Chroma(client=chroma_client, embedding_function=embeddings_model, 
                  collection_name="hpo_ontology_esp_FULL")
retriever = vectordb_sep.as_retriever(search_kwargs={"k": 20})

In [27]:
def process_unique_metadata(doc_array, MAX_DOCS=5):
    unique_ids = []
    i=0
    while len(unique_ids) < MAX_DOCS and i<len(doc_array):
        if doc_array[i].metadata["hpo_id"] not in unique_ids:
            unique_ids.append(doc_array[i].metadata["hpo_id"] )
        i += 1
    return ontology.get_by_ids(unique_ids)

In [28]:
doc_array = process_unique_metadata(retriever.invoke("convulsiones"))

In [29]:
doc_array

[Document(id='HP:0001250', metadata={'lineage': 'HP:0012638->HP:0000707->HP:0000118', 'hpo_id': 'HP:0001250'}, page_content='Convulsiones. Epilepsia. Crisis epiléptica. Convulsiones. Una crisis epiléptica es una anomalía intermitente de la fisiología del sistema nervioso caracterizada por la aparición transitoria de signos y/o síntomas debidos a una actividad neuronal anormal excesiva o sincrónica en el cerebro.'),
 Document(id='HP:0007359', metadata={'lineage': 'HP:0001250->HP:0012638->HP:0000707->HP:0000118', 'hpo_id': 'HP:0007359'}, page_content='Convulsiones focales. Crisis de inicio focal. Convulsión focal. Convulsiones focales. Crisis de inicio focal. Convulsión parcial. Crisis parciales. Crisis que afecta a una mitad del cerebro. Una crisis de inicio focal es un tipo de crisis que se origina en redes limitadas a un hemisferio. Pueden estar discretamente localizadas o más ampliamente distribuidas, y pueden originarse en estructuras subcorticales.'),
 Document(id='HP:0011097', met

In [19]:
with open(os.path.join(PROJECT_DIR, "./resources/keyword_retriever.pkl"), "rb") as fp:
    keyword_retriever = pkl.load(fp)

In [20]:
ensemble_retriever = EnsembleRetriever(retrievers=[vectordb.as_retriever(),
                                                   keyword_retriever],
                                       weights=[0.6, 0.4], id_key="hpo_id", k=4)

In [11]:
# prompt = """Revisa cuidadosamente cada frase de la nota clínica para identificar términos relacionados con patrones de herencia genética, anomalías anatómicas, síntomas clínicos, hallazgos diagnósticos, resultados de pruebas y afecciones o síndromes específicos.
# Ignora por completo los hallazgos negativos, los hallazgos normales (es decir, «normal» o «no»), los procedimientos y los antecedentes familiares. Incluye el contexto apropiado basándote únicamente en el pasaje.
# Devuelve los términos extraídos en un objeto JSON con una única clave 'fenotipos', que contiene la lista de términos extraídos escritos correctamente. Asegúrate de que el resultado sea conciso, sin notas, comentarios ni metaexplicaciones adicionales. No dejes fuera adjetivos críticos para ese fenotipo.
# <<NOTA CLÍNICA>>>
# {clinical_note}"""
# prompt = PromptTemplate.from_template(prompt)

Definir el output

In [21]:
system =  """Eres un experto de codificación de fenotipos de la ontología Human Phenotype Ontology. Para ello primero debes determinar qué fenotipos están presentes en la nota clínica. Sigue los siguientes pasos: 
1. A partir del siguiente texto clínico, identifica términos del texto que sugieran fenotipos clínicos relevantes, incluyendo diagnósticos, síntomas, signos físicos y hallazgos de laboratorio. 
2. Ignora por completo los hallazgos negativos, los hallazgos normales (es decir, «normal» o «no»), los procedimientos y los antecedentes familiares. 
3. Si algún valor incluye de forma implícita un fenotipo, infiérelo y menciónalo como tal en el campo "phenotype".
4. Si el valor no permite inferir con seguridad un fenotipo, simplemente describe el resultado de la analítica en lenguaje natural.
5. Para cada término, a parte del fenotipo, incluye la frase a la que pertenezca en la nota clínica original.
6. Sé específico, cada término debe contener un solo fenotipo asociado. Si tiene dos fenotipos, duplícalo y menciona ambos fenotipos. 
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{clinical_note}"),
    ]
)

In [22]:
class PhenotypeCandidate(BaseModel):
    """Fenotipos, patrones de herencia genética, anomalías anatómicas, síntomas clínicos, hallazgos diagnósticos, resultados de pruebas y afecciones o síndromes específicos en la nota clínica"""
    extract: str = Field(description="Un único fenotipo, diagnóstico, sintoma clínico, anomalia anatómica o prueba de laboratorio")
    phenotype: str = Field(description="Nombre del posible fenotipo asociado en español")
    context: str = Field( description="Parte de la frase en el que se menciona el extracto.")


class Data(BaseModel):
    """Información extraída sobre los fenotipos encontrados y los términos y contexto en el que se encuentran"""
    candidates: List[PhenotypeCandidate]

In [23]:
structured_llm = llm.with_structured_output(Data, method="json_schema")

In [25]:
extractphenotypes = prompt | structured_llm
answer = extractphenotypes.invoke(
    {
        "clinical_note": clinical_note,
    }
)

In [17]:
with open("../../resources/names_dict.pkl", "rb") as fp:
    names_dict = pkl.load(fp)

In [26]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

sys_template = """Identifica el término de la Ontología de Fenotipos Humanos (HPO) más apropiado para cada extracto de las notas clínicas del paciente a partir de una lista de candidatos (Código HPO - Descripción).
Da prioridad a los términos que sean concisos y directamente pertinentes para el síntoma o afección principal descritos. 
Céntrate en el tema central de cada frase y evita seleccionar opciones con detalles descriptivos o situacionales adicionales a menos que sean esenciales para captar con precisión el fenotipo. 
Asegúrate de que el término HPO elegido coincide estrechamente con la afección del paciente tal como se describe, sin añadir términos nuevos o extraños. 
Si hay varios candidatos, selecciona y devuelve el término HPO más pertinente que mejor represente la afección o síntoma primario. Proporciona sólo los códigos HPO elegidos. La nota clínica original es la siguiente:
{clinical_note}
"""

human_template =  """Término: {term} ({phenotype})
Contexto: {context}
Candidatos: {candidates}"""

chat_template = ChatPromptTemplate.from_messages([
    ('system', sys_template),
    ('human', human_template)
])

# mesages = chat_template.format_messages(term="Bob", context="What is your name?", candidates="hey")

In [27]:
class HpoCode(BaseModel):
    """Código HPO asignado con el formato HP:#######"""
    hpo_code: str

In [28]:
llm_output = llm.with_structured_output(HpoCode)

In [21]:
hpo_assignment = chat_template | llm_output

In [49]:
vectordb.similarity_search("Creencia delirante de olor corporal sudoroso", k=5)

[Document(id='HP:0410021', metadata={'hpo_id': 'HP:0410021', 'lineage': 'HP:0500001->HP:0025142->HP:0000118'}, page_content='Olor a humedad. Olor a humedad. Olor corporal penetrante.'),
 Document(id='HP:0000975', metadata={'hpo_id': 'HP:0000975', 'lineage': 'HP:0007550->HP:0025276->HP:0001574->HP:0000118'}, page_content='Hiperhidrosis. Diaforesis. Sudoración excesiva. Aumento de la sudoración. Sudoración profusa. Sudando. Sudoración profusa. Aumento de la sudoración. Transpiración excesiva anormal (sudoración) a pesar de la falta de estímulos apropiados como el clima cálido y húmedo.'),
 Document(id='HP:0500001', metadata={'hpo_id': 'HP:0500001', 'lineage': 'HP:0025142->HP:0000118'}, page_content='Olor corporal. BO. Olor corporal. Bromhidrosis. Bromidrosis. Osmidrosis. Olor desagradable percibido que desprende el cuerpo.'),
 Document(id='HP:5200200', metadata={'lineage': 'HP:0000738->HP:5200423->HP:0011446->HP:0012638->HP:0000707->HP:0000118', 'hpo_id': 'HP:5200200'}, page_content='Alu

In [36]:
new_docs = retriever.invoke(answer.candidates[0].extract)
[doc.id for doc in new_docs]

['HP:0020060', 'HP:0011895', 'HP:0001903']

In [40]:
from utils.fuzzyretriever import FuzzyRetriever
fuzzyretriever = FuzzyRetriever()

In [41]:
def pretty_print_candidates(docs):
    final_str = ""
    for doc in docs: 
        final_str += f"{doc.id} - {doc.page_content}\n"
    return final_str

In [24]:
from langchain_core.runnables import chain
from langsmith import traceable

@chain
def custom_chain(question):
    response = extractphenotypes.invoke(question)
    docs = []
    intermediate_results = []
    for query in response.candidates:
        new_docs = retriever.invoke(query.context)
        fuzzy_docs = [x[0] for x in fuzzyretriever.invoke(query.phenotype)]
        new_docs =  vectordb.get_by_ids(set(fuzzy_docs)) + new_docs if len(set(fuzzy_docs)) > 0 else new_docs
        docs.append({"clinical_note":question,
                     "term":query.extract,
                     "phenotype":query.phenotype,
                     "context":query.context,
                     "candidates":pretty_print_candidates(new_docs)})
        intermediate_results.append([doc.id for doc in new_docs])    
    answer = hpo_assignment.batch(docs)
    return {"final answer": answer, "docs": intermediate_results}

In [42]:
import importlib
import utils.customchain as cc
custom_chain = cc.custom_chain

In [43]:
from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler(
    public_key="pk-lf-4c4b9492-d7ca-4f6d-b7af-325ea09726c0",
    secret_key="sk-lf-1b2e42c8-bf1e-4150-bf36-fce439ad2270",
    host="http://localhost:3000"
)

In [44]:
answer = custom_chain.invoke({"clinical_note":clinical_note})

In [45]:
answer = custom_chain.with_config(
    {
        "run_name": "fuzzy matching",
        "metadata": {"version": "v1", "owner": "mdiazrio"},
        "callbacks": [langfuse_handler]
    }
).invoke({"clinical_note":clinical_note})

Raw GPT

In [ ]:
system = """Eres una herramienta que sirve para extraer fenotipos de la ontología Human Phenotype Ontolgy a partir de notas clínicas para ello: 
1. A partir del siguiente texto clínico, identifica todos los fenotipos clínicos relevantes, incluyendo diagnósticos, síntomas, signos físicos y hallazgos de laboratorio. 
2. Ignora por completo los hallazgos negativos, los hallazgos normales (es decir, «normal» o «no»), los procedimientos y los antecedentes familiares. 
3. Si algún valor incluye de forma implícita un fenotipo, infiérelo y menciónalo como tal.
4. Para cada término, asigna el código HPO apropiado. 
Devuelve un JSON con la llave "final_answer" y el listado de códigos HPO detectados.
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{clinical_note}"),
    ]
)

In [24]:
import json
from langchain_core.runnables import chain

init_chain = prompt | llm

@chain
def rawgptchain(question):
    response = init_chain.invoke(question)
    return json.loads(response.content)

In [31]:
import sys
sys.path.append('../utils')
import importlib
import rawgptchain as rgp

In [55]:
importlib.reload(rgp)
rawgptchain = rgp.rawgptchain

In [56]:
response = rawgptchain.with_config({"callbacks": [langfuse_handler]}).invoke({"clinical_note":clinical_note})

FlashRank

In [10]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

In [11]:
from flashrank import Ranker 

ranker = Ranker(model_name="ms-marco-MiniLM-L-12-v2")

In [16]:
compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [17]:
compression_retriever.invoke(context)

[Document(metadata={'id': 8, 'relevance_score': 0.9918292, 'hpo_id': 'HP:0006780', 'lineage': 'HP:0100733->HP:0000818->HP:0011766->HP:0000828->HP:0100568->HP:0002664->HP:0011793->HP:0000118'}, page_content='Carcinoma de paratiroides. Tumor maligno de las glándulas paratiroides. El carcinoma paratiroideo suele segregar hormona paratiroidea, lo que provoca hiperparatiroidismo.'),
 Document(metadata={'id': 4, 'relevance_score': 0.78583723, 'hpo_id': 'HP:0100031', 'lineage': 'HP:0000820->HP:0000818->HP:0100568->HP:0002664->HP:0011793->HP:0000118->HP:0011772'}, page_content='Neoplasia del tiroides. Tumor (crecimiento anormal de tejido) de la glándula tiroides.'),
 Document(metadata={'id': 1, 'relevance_score': 0.60732955, 'hpo_id': 'HP:0011779', 'lineage': 'HP:0002890->HP:0100031->HP:0000820->HP:0000818->HP:0100568->HP:0002664->HP:0011793->HP:0000118->HP:0011772'}, page_content='Carcinoma anaplásico de tiroides.')]

In [18]:
reranked_docs = []
for context in [c.context for c in answer.candidates]:
    reranked_docs.append(compression_retriever.invoke(context))

In [19]:
reranked_docs

[[Document(metadata={'id': 5, 'relevance_score': 0.99929035, 'hpo_id': 'HP:0045081', 'lineage': 'HP:0004323->HP:0001507->HP:0000118'}, page_content='Anomalía del índice de masa corporal. Anomalía en la relación peso/altura al cuadrado, calculada dividiendo el peso del individuo en kilogramos por el cuadrado de la altura del individuo en metros y utilizada como indicador de obesidad e insuficiencia ponderal en comparación con las medias.'),
  Document(metadata={'id': 6, 'relevance_score': 0.99896955, 'hpo_id': 'HP:0031418', 'lineage': 'HP:0045081->HP:0004323->HP:0001507->HP:0000118'}, page_content='Aumento del índice de masa corporal. Relación peso/altura al cuadrado anormalmente elevada, calculada dividiendo el peso del individuo en kilogramos por el cuadrado de la altura del individuo en metros y utilizada como indicador de sobrepeso en comparación con las medias.'),
  Document(metadata={'id': 0, 'relevance_score': 0.99863136, 'hpo_id': 'HP:6000179', 'lineage': 'HP:0430103->HP:0032443